# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| export
import code_tokenizers
import json

from transformers import AutoTokenizer
from tree_sitter import Language, Parser

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
def unroll_node_types(
    nested_node_types: dict, # node_types from tree-sitter
) -> list: # list of node types
    """Unroll nested node types into a flat list of node types. This includes subtypes as well."""
    node_types = [node_type["type"] for node_type in nested_node_types]
    node_subtypes = [
        node_subtype["type"]
        for node_type in node_types
        if "subtypes" in node_type
        for node_subtype in node_type["subtypes"]
    ]
    return list(set(node_types + node_subtypes))

In [ ]:
#| export
# From: https://github.com/github/CodeSearchNet/tree/master/function_parser
def traverse(
    node,       # tree-sitter node
    results,    # list to append results to
) -> None:
    """Traverse in a recursive way, a tree-sitter node and append results to a list."""
    if node.type == 'string':
        results.append(node)
        return
    for n in node.children:
        traverse(n, results)
    if not node.children:
        results.append(node)

In [ ]:
#| export
def get_token_type(
    tok_span: tuple, # (start, end) position of a token
    nodes: list,     # list of tree-sitter nodes
    lines: list,     # list of lines in the code
) -> tuple: # (parent_type, token_type) of the token
    """Get the parent AST type and token AST type of a token."""
    def get_node_span(node):
        def convert_to_offset(point):
            row, column = point
            chars_in_rows = sum(map(len, lines[:row])) + row
            chars_in_columns = len(lines[row][:column])

            offset = chars_in_rows + chars_in_columns
            return offset
        start_span = convert_to_offset(node.start_point)
        end_span = convert_to_offset(node.end_point)
        return start_span, end_span
    
    node_spans = [get_node_span(node) for node in nodes]
    for i, span in enumerate(node_spans):
        if (span[0] <= tok_span[0] and tok_span[0] < span[1]) or (span[0] < tok_span[1] and tok_span[1] <= span[1]):
            return nodes[i].parent.type, nodes[i].type

In [ ]:
#| export
class CodeTokenizer():
    """A tokenizer for code, which aligns the tokens with the AST nodes."""
    def __init__(
        self,
        tokenizer,  # transformers tokenizer
        parser,     # tree-sitter parser
        node_types  # list of node types
    ):
        self.tokenizer = tokenizer
        self.parser = parser
        self.node_types = node_types
    
    def parse_tree(
        self,
        code,           # code to parse
        offset_mapping  # offset mapping from the tokenizer to align the tokens with the AST nodes
    ):                  # returns a list of AST ids and a list of parent AST ids
        tree = self.parser.parse(bytes(code, "utf8"))
        nodes = []
        traverse(tree.root_node, nodes)

        ast_ids = []
        parent_ast_ids = []
        for i, (start, end) in enumerate(offset_mapping):
            if start == None or end == None:
                ast_ids.append(-1)
                parent_ast_ids.append(-1)
                continue
            type_info = get_token_type((start, end), nodes, code.split("\n"))
            if type_info is None:
                ast_ids.append(-1)
                parent_ast_ids.append(-1)
            else:
                parent_node_type, node_type = type_info
                try:
                    ast_ids.append(self.node_types.index(node_type))
                    parent_ast_ids.append(self.node_types.index(parent_node_type))
                except Exception as e:
                    print(type_info)
                    print(code)
                    ast_ids.append(-1)
                    parent_ast_ids.append(-1)
                    raise e

        return ast_ids, parent_ast_ids
    
    def __call__(
        self,
        code,               # code or list of code to tokenize
        return_merged=True, # whether to string representations of the merged ASTs and parent ASTs
        **kwargs            # kwargs for the underlying transformers tokenizer
    ):                      # returns a dictionary of token ids, attention masks, AST ids, parent AST ids, and optionally the string representations of the merged ASTs and parent ASTs
        encoding = self.tokenizer(code, return_offsets_mapping=True, **kwargs)
        if isinstance(code, list):
            encoding["ast_ids"] = []
            encoding["parent_ast_ids"] = []
            for i, c in enumerate(code):
                ast_ids, parent_ast_ids = self.parse_tree(c, encoding["offset_mapping"][i])
                encoding["ast_ids"].append(ast_ids)
                encoding["parent_ast_ids"].append(parent_ast_ids)
        else:
            encoding["ast_ids"], encoding["parent_ast_ids"] = self.parse_tree(code, encoding["offset_mapping"])
        
        if return_merged:
            # Merge the AST ids with their parent AST ids and use the names instead of the ids
            if isinstance(code, list):
                encoding["merged_ast"] = []
                for parent_ast_ids, ast_ids in zip(encoding["parent_ast_ids"], encoding["ast_ids"]):
                    merged_ast = []
                    for i, (parent_ast_id, ast_id) in enumerate(zip(parent_ast_ids, ast_ids)):
                        if parent_ast_id == -1 or ast_id == -1:
                            merged_ast.append("< N/A >")
                        else:
                            merged_ast.append(
                                f"<{self.node_types[parent_ast_id]} -> {self.node_types[ast_id]}>"
                            )

                    encoding["merged_ast"].append(merged_ast)
            else:
                encoding["merged_ast"] = []
                for parent_ast_id, ast_id in zip(encoding["parent_ast_ids"], encoding["ast_ids"]):
                    if parent_ast_id == -1 or ast_id == -1:
                        encoding["merged_ast"].append("< N/A >")
                    else:
                        encoding["merged_ast"].append(
                            f"<{self.node_types[parent_ast_id]} -> {self.node_types[ast_id]}>"
                        )

        return encoding
    
    def decode(self, *args, **kwargs):
        return self.tokenizer.decode(*args, **kwargs)

    @staticmethod
    def from_pretrained(
        name_or_path: str,  # name or path of the tokenizer
        lang: str,          # language of the tokenizer
    ):                      # CodeTokenizer for the given language
        """Create a CodeTokenizer from a pretrained tokenizer for a given language."""
        tokenizer = AutoTokenizer.from_pretrained(name_or_path)

        # Grab the node types from the tree-sitter language
        language = Language(f"{code_tokenizers.__path__[0]}/grammars/tree-sitter-languages.so", lang)
        node_path = f"{code_tokenizers.__path__[0]}/grammars/tree-sitter-{lang}/src/node-types.json"
        with open(node_path) as f:
            node_types = json.load(f)
        node_types = unroll_node_types(node_types)
        if lang == "python":
            node_types.append("as_pattern_target")

        # Create a parser for the language
        parser = Parser()
        parser.set_language(language)
        
        return CodeTokenizer(tokenizer, parser, node_types)

In [ ]:
# test the tokenizer
py_tokenizer = CodeTokenizer.from_pretrained("gpt2", "python")
code = "def foo():\n    print('hello world')"

encoding = py_tokenizer(code)

assert "ast_ids" in encoding
assert "parent_ast_ids" in encoding
assert "merged_ast" in encoding
assert len(encoding["ast_ids"]) == len(encoding["input_ids"])
assert len(encoding["parent_ast_ids"]) == len(encoding["input_ids"])
assert len(encoding["merged_ast"]) == len(encoding["input_ids"])

In [ ]:
# test with list of code
code = ["def foo():\n    print('hello world')", "def bar():\n    print('hello world')"]
encoding = py_tokenizer(code)

assert "ast_ids" in encoding
assert "parent_ast_ids" in encoding
assert "merged_ast" in encoding
assert len(encoding["ast_ids"]) == len(encoding["input_ids"])
assert len(encoding["parent_ast_ids"]) == len(encoding["input_ids"])
assert len(encoding["merged_ast"]) == len(encoding["input_ids"])
assert len(encoding["ast_ids"][0]) == len(encoding["input_ids"][0])
assert len(encoding["parent_ast_ids"][0]) == len(encoding["input_ids"][0])
assert len(encoding["merged_ast"][0]) == len(encoding["input_ids"][0])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()